In [1]:
import torch

assert torch.cuda.is_available()

In [2]:
import os
from drugex.training.generators import SequenceRNN
from drugex.data.corpus.vocabulary import VocSmiles

MODELS_PR_PATH = "data/models/pretrained/smiles-rnn/Papyrus05.5_smiles_rnn_PT/"

voc = VocSmiles.fromFile(os.path.join(MODELS_PR_PATH, "Papyrus05.5_smiles_rnn_PT.vocab"), encode_frags=False)
pretrained = SequenceRNN(voc, is_lstm=True)
pretrained.loadStatesFromFile(os.path.join(MODELS_PR_PATH, "Papyrus05.5_smiles_rnn_PT.pkg"))

/home/sichom/software/miniconda/envs/8add-olomouc-sicho/lib/python3.12/site-packages/drugex/training/interfaces.py:322: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.lo

In [3]:
generated_pretrained = pretrained.generate(num_samples=100)
generated_pretrained

Generating molecules:   0%|          | 0/100 [00:00<?, ?it/s]

,SMILES,Valid
0,CCn1nnnc1SCC(=O)Nc1cccc(NC(=O)c2ccccc2)c1,1.0
1,COc1cc(C=C2SC(=S)NC2=O)ccc1Oc1ccc([N+](=O)[O-]...,1.0
2,CC1COCC2CSC(c3ccccc3)N2S1,1.0
3,CC1(C)CC1C(=O)NC(=CCC(F)(F)F)C(=O)O,1.0
4,CC(NS(=O)(=O)c1ccc(Cl)cc1)C(=O)Nc1cccc([N+](=O...,1.0
...,...,...
95,O=C(NCc1ccc(S(=O)(=O)C2CCCCC2)cc1)N1Cc2ccncc2C1,1.0
96,CS(=O)(=O)c1ccc(-n2nc(C(F)F)cc2-c2ccc(Cl)cc2)cc1,1.0
97,Nc1nc(O)c2ncn(COC(=O)NCCCNc3nn[nH]n3)c2n1,1.0
98,Nc1nc(C(=O)NCc2cccc3cccnc23)c2cccc(C(F)(F)F)c2n1,1.0


In [4]:
from qsprpred.data import MoleculeTable

generated_pretrained = MoleculeTable(
    name="GeneratedPretrained", 
    df=generated_pretrained
)
len(generated_pretrained)

100

In [5]:
from qsprpred.data.descriptors.fingerprints import MorganFP

morgan_fp = MorganFP(3, 2048)
generated_pretrained.addDescriptors([morgan_fp])
generated_pretrained.getDescriptors().head()

,MorganFP_MorganFP_0,MorganFP_MorganFP_1,MorganFP_MorganFP_2,MorganFP_MorganFP_3,MorganFP_MorganFP_4,MorganFP_MorganFP_5,MorganFP_MorganFP_6,MorganFP_MorganFP_7,MorganFP_MorganFP_8,MorganFP_MorganFP_9,...,MorganFP_MorganFP_2038,MorganFP_MorganFP_2039,MorganFP_MorganFP_2040,MorganFP_MorganFP_2041,MorganFP_MorganFP_2042,MorganFP_MorganFP_2043,MorganFP_MorganFP_2044,MorganFP_MorganFP_2045,MorganFP_MorganFP_2046,MorganFP_MorganFP_2047
QSPRID,,,,,,,,,,,,,,,,,,,,,
GeneratedPretrained_000,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
GeneratedPretrained_001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
GeneratedPretrained_002,False,False,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
GeneratedPretrained_003,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
GeneratedPretrained_004,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
from scaffviz.clustering.manifold import UMAP
from scaffviz.depiction.plot import Plot

plt = Plot(UMAP(n_neighbors=5, min_dist=0.1, metric="jaccard", random_state=42))
plt.plot(
    generated_pretrained,
    color_by='Valid',
    recalculate=True,
    port=9000,
    viewport_height=800
)

/home/sichom/software/miniconda/envs/8add-olomouc-sicho/lib/python3.12/site-packages/umap/umap_.py:1887: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/home/sichom/software/miniconda/envs/8add-olomouc-sicho/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dash is running on http://127.0.0.1:9000/

